In [1]:
import findspark
findspark.init()

from pyspark import *
from pyspark.sql.functions import desc, col, rand
from pyspark.sql import *
from graphframes import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import sys
from sympy.ntheory.generate import nextprime
import time

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# https://graphframes.github.io/graphframes/docs/_site/quick-start.html
# https://stackoverflow.com/questions/65011599/how-to-start-graphframes-on-spark-on-pyspark-on-juypter-on-docker
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'
os.environ['PYSPARK_PYTHON'] = sys.executable

In [4]:
spark = SparkSession.builder.appName('hw2') \
        .master("local[2]").getOrCreate()

21/11/17 23:39:51 WARN Utils: Your hostname, mark-machine resolves to a loopback address: 127.0.1.1; using 130.237.2.47 instead (on interface wlp8s0)
21/11/17 23:39:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mark/.ivy2/cache
The jars for the packages stored in: /home/mark/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9d4c4ac-969a-4746-979a-0d70468b2ef7;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 116ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------

In [5]:
data_path = "data/T10I4D100K.dat"

In [6]:
def read_data(data_path):
    # read raw data
    data_raw_rdd = spark.sparkContext.textFile(data_path)
    # transform raw data to list of lists (list of ordered baskets)
    data_rdd = data_raw_rdd.map(lambda x: [int(y) for y in x.strip().split(" ")])
    # get item counts (k:v where k is hashed item id and v is count)
    items_counts_rdd = data_rdd.flatMap(lambda list: list).map(lambda w: (w,1)).reduceByKey(lambda a, b: a+b)
    # get items
    items_rdd = items_counts_rdd.map(lambda x: x[0])
    
    return data_rdd, items_rdd, items_counts_rdd

In [7]:
data_rdd, items_rdd, items_counts_rdd = read_data(data_path=data_path)

In [8]:
print(data_rdd.take(2))
n_baskets = data_rdd.count()
print(f"n_baskets = {n_baskets}")

[[25, 52, 164, 240, 274, 328, 368, 448, 538, 561, 630, 687, 730, 775, 825, 834], [39, 120, 124, 205, 401, 581, 704, 814, 825, 834]]
n_baskets = 100000


In [9]:
items_rdd.take(20)
n_items = items_rdd.count()
print(f"n_items = {n_items}")

n_items = 870


In [10]:
print(items_counts_rdd.count())
items_counts_rdd.take(10)

870


[(52, 1983),
 (164, 744),
 (240, 1399),
 (274, 2628),
 (328, 663),
 (368, 7828),
 (448, 1370),
 (538, 3982),
 (630, 1523),
 (730, 602)]

In [11]:
import itertools

def get_singletons(items_counts_rdd, s):
    singletons_rdd = items_counts_rdd.filter(lambda x: s <= x[1])
    singletons_rdd = singletons_rdd.map(lambda x: ([x[0]], x[1]))
    return singletons_rdd

def construct_itemsets(k, itemsets_frequent_rdd):
    assert 1 < k
    singletons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == 1)
    k_minus_1_tons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == k-1)
    k_minus_1_tons_rdd_collected = k_minus_1_tons_rdd.collect()
    # make tuple of itemsets of size k
    # return itemsets_candidates_rdd

def filter_itemsets(itemsets_candidates_rdd, k, s):
    pass
    # for each itemset of size k (itemset_k) in itemsets of size k, for each basket in baskets, 
    # if intersection of itemset_k and basket is equal to k, support counter of itemset_k +1
    
    # for itemset_k in itemset_ks, if support counter of itemset_k is >= s, keep, otherwise drop

In [12]:
itemsets_frequent_rdd = spark.sparkContext.parallelize([])

In [13]:
itemsets_frequent_rdd.collect()

[]

In [14]:
s = 1000
itemsets_frequent_rdd = get_singletons(items_counts_rdd=items_counts_rdd, s=s)
itemsets_frequent_rdd.count()

375

In [15]:
itemsets_frequent_rdd.take(10)

[([52], 1983),
 ([240], 1399),
 ([274], 2628),
 ([368], 7828),
 ([448], 1370),
 ([538], 3982),
 ([630], 1523),
 ([834], 1373),
 ([120], 4973),
 ([704], 1794)]

In [16]:
k = 2
singletons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == 1)
k_minus_1_tons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == k-1)

In [17]:
k_minus_1_tons_rdd.take(10)

[([52], 1983),
 ([240], 1399),
 ([274], 2628),
 ([368], 7828),
 ([448], 1370),
 ([538], 3982),
 ([630], 1523),
 ([834], 1373),
 ([120], 4973),
 ([704], 1794)]

In [18]:
l1 = singletons_rdd.map(lambda x: x[0])
l2 = k_minus_1_tons_rdd.map(lambda x: x[0])

In [19]:
l3 = l1.cartesian(l2)
l3.take(10)

[([52], [52]),
 ([52], [240]),
 ([52], [274]),
 ([52], [368]),
 ([52], [448]),
 ([52], [538]),
 ([52], [630]),
 ([52], [834]),
 ([52], [120]),
 ([52], [704])]

In [20]:
l4 = l3.map(lambda x: [*x[0], *x[1]])

In [21]:
l4.take(10)

[[52, 52],
 [52, 240],
 [52, 274],
 [52, 368],
 [52, 448],
 [52, 538],
 [52, 630],
 [52, 834],
 [52, 120],
 [52, 704]]

In [22]:
l5 = l4.filter(lambda x: len(set(x)) == 2)

In [23]:
l5.take(10)

[[52, 240],
 [52, 274],
 [52, 368],
 [52, 448],
 [52, 538],
 [52, 630],
 [52, 834],
 [52, 120],
 [52, 704],
 [52, 814]]

In [24]:
l5.take(10)

[[52, 240],
 [52, 274],
 [52, 368],
 [52, 448],
 [52, 538],
 [52, 630],
 [52, 834],
 [52, 120],
 [52, 704],
 [52, 814]]

In [25]:
l5c = l5.count()

In [26]:
def count2(candidate, basket, k):
    intersections_all = list(set(basket) & set(candidate))
    return 1 if len(intersections_all) == k else 0

def count(candidate, data_rdd_collected, k):
    intersections_all = map(lambda x: list(set(x) & set(candidate)), data_rdd_collected)
    intersections = list(filter(lambda x: len(x) == 2, intersections_all))
    support = len(intersections)
    return support

data_rdd_collected = data_rdd.collect()

In [ ]:
data_rdd_br = spark.sparkContext.broadcast(data_rdd.collect())
#data_rdd_br.value
l5.cache()

l6 = l5.map(lambda x: count(x,data_rdd_collected, 2))
#l6 = l5.map(lambda x: sum([count2(x, data_rdd_br.value[i], 2) for i in range(n_baskets)]))

In [ ]:
l6.take(1000)

In [ ]:
l5_collected = l5.collect()

In [ ]:
c = l5_collected[1]
c

In [ ]:
intersections = data_rdd.map(lambda x: list(set(x) & set(c)))

In [ ]:
intersections.count()

In [ ]:
intersections.count()

In [ ]:
intersections_nonnull = intersections.filter(lambda x: x)
s = intersections_nonnull.count()

In [ ]:
intersections_nonnull.take(10)

In [ ]:
s

In [ ]:
intersections = map(lambda x: list(set(x) & set(c)), data_rdd_collected)
yya = len(list(filter(lambda x: x, intersections)))

In [ ]:
yya

In [ ]:
intersections = data_rdd.map(lambda x: list(set(x) & set(c))).filter(lambda x: len(x) == 2)
intersections.count()

In [ ]:
intersections.take(40)

In [ ]:
intersections = list(filter(lambda x: len(x) == 2, map(lambda x: list(set(x) & set(c)), data_rdd_collected)))

In [ ]:
len(intersections)

In [30]:
l6 = spark.sparkContext.parallelize([i for i in range(l5c)])
l5_collected = l5.collect()

In [39]:
l6.cartesian(data_rdd).map(lambda x: (x[0], len(set(l5_collected[x[0]]) & set(x[1])) == 2)).groupBy(lambda x: x[0]).take(1)

KeyboardInterrupt: 

In [32]:
l5.take(20)

[[52, 240],
 [52, 274],
 [52, 368],
 [52, 448],
 [52, 538],
 [52, 630],
 [52, 834],
 [52, 120],
 [52, 704],
 [52, 814],
 [52, 674],
 [52, 854],
 [52, 950],
 [52, 422],
 [52, 964],
 [52, 294],
 [52, 708],
 [52, 738],
 [52, 766],
 [52, 966]]

In [33]:
l5_collected

[[52, 240],
 [52, 274],
 [52, 368],
 [52, 448],
 [52, 538],
 [52, 630],
 [52, 834],
 [52, 120],
 [52, 704],
 [52, 814],
 [52, 674],
 [52, 854],
 [52, 950],
 [52, 422],
 [52, 964],
 [52, 294],
 [52, 708],
 [52, 738],
 [52, 766],
 [52, 966],
 [52, 978],
 [52, 104],
 [52, 620],
 [52, 798],
 [52, 214],
 [52, 350],
 [52, 658],
 [52, 682],
 [52, 782],
 [52, 970],
 [52, 390],
 [52, 192],
 [52, 208],
 [52, 280],
 [52, 496],
 [52, 530],
 [52, 618],
 [52, 720],
 [52, 914],
 [52, 932],
 [52, 276],
 [52, 706],
 [52, 878],
 [52, 424],
 [52, 490],
 [52, 910],
 [52, 960],
 [52, 130],
 [52, 392],
 [52, 862],
 [52, 78],
 [52, 900],
 [52, 572],
 [52, 778],
 [52, 266],
 [52, 290],
 [52, 458],
 [52, 614],
 [52, 888],
 [52, 944],
 [52, 70],
 [52, 204],
 [52, 334],
 [52, 480],
 [52, 874],
 [52, 504],
 [52, 890],
 [52, 310],
 [52, 722],
 [52, 810],
 [52, 844],
 [52, 846],
 [52, 918],
 [52, 326],
 [52, 526],
 [52, 774],
 [52, 788],
 [52, 116],
 [52, 198],
 [52, 946],
 [52, 638],
 [52, 678],
 [52, 780],
 [52, 

In [31]:
l6.take(20)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
l5_collected